In [51]:
# Workspace problem with several narrow gaps

import torch
import torch.nn as nn

from functools import partial
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree as KDTree
import sys
import time
import cv2

sys.path.append('/home/oscar_palfelt/MSc_thesis/ompl/py-bindings')
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og

cuda = True
DEVICE = torch.device("cuda" if cuda else "cpu")

In [52]:
# neural network parameters
mb_size = 256 # mini batch dim
h_Q_dim = 512*2 # encoder dim
h_P_dim = 512*2 # decoder dim

c = 0 # unsure of this constant
lr = 1e-4 # learning rate

# problem dimenc_dimsions
nDrawnSamples = 1 # number of dependent samples to draw during smapling
dim = 4 # (x, y, xdot, ydot)
dataElements = nDrawnSamples * dim + 2 * dim # sample (4D), init (4D), goal (4D)

z_dim = 2 # latent dim
X_dim = dim*nDrawnSamples # samples dim
y_dim = dim # reconstruction of the original point (unsused?)
c_dim = dataElements - dim # dimension of conditioning variable

In [53]:
# read in data from .txt file, re-arrange to allow drawing multiple depedent samples

filename = '/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/motion_planning/data/pathDataDifficultTest'
rawdata = np.genfromtxt(filename, delimiter=',', dtype='d')
_, pathsIdx = np.unique(rawdata[:,4:], axis=0, return_index=True)
pathsIdx.sort()

pathsLenghts = pathsIdx[1:] - pathsIdx[:-1]
validLengthsIdx = np.argwhere(pathsLenghts >= nDrawnSamples)
validPlansIdx = pathsIdx[validLengthsIdx]

data = np.zeros(shape=(1, dim*nDrawnSamples + dim*2))

for c, i in enumerate(validPlansIdx.reshape(-1)):
    rndSample = np.random.choice(np.arange(pathsLenghts[c]), size=(np.floor(pathsLenghts[c] / nDrawnSamples).astype(int), nDrawnSamples), replace=False)
    rndSample.sort(axis=1)

    for sample in rndSample:
        data = np.vstack((data,np.append(rawdata[i + sample, :4].reshape(1, dim*nDrawnSamples), rawdata[i, 4:].reshape(1, dim*2), axis=1))) 

numEntries = data.shape[0]
print(numEntries)

74262


In [54]:
# generate conditioning variable
filename = '/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/motion_planning/data_generation/map_difficult.png'
mapImg = cv2.imread(filename, 0)
blurImg = cv2.blur(mapImg, ksize=(3,3))
occGrid = np.clip(mapImg, 0, 1)
#occGrid = np.floor(blurImg / 255)

assert occGrid.shape[0] == occGrid.shape[1]
gridSize = occGrid.shape[0]

conditionsOcc = np.tile(occGrid.reshape(1, gridSize ** 2), reps=(numEntries,1))
    
#cs = np.concatenate((data[0:numEntries,dim:dataElements], conditionsOcc), axis=1) # init (4D), goal (4D), occ(30x30)
cs = data[0:numEntries,dim*nDrawnSamples:dataElements]

In [55]:
# define pytorch networks
# based on https://github.com/Jackson-Kang/Pytorch-VAE-tutorial/blob/master/.ipynb_checkpoints/01_Variational_AutoEncoder-checkpoint.ipynb

class Encoder(nn.Module):
    
    def __init__(self, input_dim=X_dim+c_dim, hidden_dim=h_Q_dim, latent_dim=z_dim):
        super(Encoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        self.z_mu = nn.Linear(hidden_dim, latent_dim)
        self.z_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x):

        seq = self.network(x)

        return self.z_mu(seq), self.z_logvar(seq)


class Decoder(nn.Module):
    def __init__(self, latent_dim=z_dim+c_dim, hidden_dim=h_P_dim, output_dim=X_dim):
        super(Decoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        
        return self.network(x)


class NeuralNetwork(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(NeuralNetwork, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
    
    def reparameterization(self, mean, logvar):
        epsilon = torch.randn_like(mean).to(DEVICE)        # sampling epsilon        
        z = mean + torch.exp(0.5 * logvar) * epsilon       # reparameterization trick
        return z

    def forward(self, x, c, encode=True):
        if encode:
            z_mu, z_logvar = self.Encoder(torch.cat((x, c), dim=1))
            z = self.reparameterization(z_mu, z_logvar)

            y = self.Decoder(torch.cat((z, c), dim=1))
            
            return y, z_mu, z_logvar
        else:
            z = x
            y = self.Decoder(torch.cat((z, c), dim=1))    

            return y

network = torch.load('/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_distributions/network_difficult.pt')

In [162]:
# define planning problem

def getThresholdPathLengthObj(si):
     obj = ob.PathLengthOptimizationObjective(si)
     obj.setCostThreshold(ob.Cost(4.0))
     return obj


class MyStateSampler(ob.StateSampler):
    def __init__(self, space):
         super(MyStateSampler, self).__init__(space)
         self.name_ = "my sampler"
         self.bounds = space.getBounds()

    def sampleUniform(self, state):
        if np.random.uniform() < p:
            rndState = y[np.random.randint(low=0, high=y.shape[0]-1), :]
            state.setYaw(float(int(rndState[3] > 0) * np.arccos(np.dot(rndState[2:4] / np.linalg.norm(rndState[2:4]), [1, 0]))))
        else:
            rndState = [np.random.uniform(low=self.bounds.low[0], high=self.bounds.high[0]),
                        np.random.uniform(low=self.bounds.low[1], high=self.bounds.high[1]),
                        np.random.uniform(low=-np.pi, high=np.pi)]
            state.setYaw(rndState[2])

        state.setX(float(rndState[0]))
        state.setY(float(rndState[1]))


def allocStateSampler(space):
    return MyStateSampler(space)


def isStateValid(spaceInformation, state):
    # perform collision checking or check if other constraints are
    # satisfied
    u = int(np.floor(state.getX() * occGrid.shape[1])) # right pointing image axis
    v = int(np.floor(occGrid.shape[0] * (1 - state.getY()))) # down pointing image axis

    if spaceInformation.satisfiesBounds(state):
        return occGrid[v,u] > 0


def problemDef(useNN=False):
    # construct the state space we are planning in
    #space = ob.DubinsStateSpace(turningRadius=0.07)
    space = ob.SE2StateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(0.001)
    bounds.setHigh(0.999)
    space.setBounds(bounds)

    # set state sampler
    if useNN:
        space.setStateSamplerAllocator(ob.StateSamplerAllocator(allocStateSampler))

    # define a simple setup class
    ss = og.SimpleSetup(space)
    ss.setStateValidityChecker(ob.StateValidityCheckerFn( \
        partial(isStateValid, ss.getSpaceInformation())))

    si = ss.getSpaceInformation()
    planner = og.RRTstar(si)
    ss.setPlanner(planner)

    ss.getProblemDefinition().setOptimizationObjective(getThresholdPathLengthObj(si))
    
    return ss


def plan(planObj, initState, goalState):

    #space = ob.DubinsStateSpace(turningRadius=0.07)
    space = ob.SE2StateSpace()
    
    start = ob.State(space)
    start().setX(initState[0])
    start().setY(initState[1])
    start().setYaw(initState[2])

    goal = ob.State(space)
    goal().setX(goalState[0])
    goal().setY(goalState[1])
    goal().setYaw(goalState[2])
    
    planObj.setStartAndGoalStates(start, goal, 0.1)

    timeTerminationCondition = ob.timedPlannerTerminationCondition(10.0)
    exactSolTerminationCondition = ob.exactSolnPlannerTerminationCondition(planObj.getProblemDefinition())
    #costTerminationCondition = ob.CostConvergenceTerminationCondition(planObj.getProblemDefinition(), epsilon=0.02)

    solved = planObj.solve(ob.plannerOrTerminationCondition(exactSolTerminationCondition, timeTerminationCondition))

    # if solved:
    #     planObj.simplifySolution()

In [ ]:
# compare performance of planning w/ and w/out the learned distributions

nTests = 10

nSamples = 2000 # number of samples to draw in latent space
p = 0.5 # likelihood which to sample from NN samples (when useNN is True)
minCostThreshold = 0.0 # set minimumcost threshold to only attempt complex planning scenarios

pdefs = [problemDef(useNN=True), problemDef(useNN=False)]

nIterations = np.zeros(shape=(2, nTests))
pathCosts = np.zeros(shape=(2, nTests))

nCompletedTests = 0

plotPaths = True
while nCompletedTests <= nTests - 1:

    planIdx = np.random.randint(0,numEntries-1); # chose a random test scenario

    c_sample_seed = cs[planIdx,:]
    c_sample = torch.from_numpy(np.repeat([c_sample_seed],nSamples,axis=0)).float().to(DEVICE)

    # directly sample from the latent space
    y = network(torch.randn(nSamples, z_dim).to(DEVICE), c_sample, encode=False).cpu().detach().numpy()

    start = data[planIdx, 4:8] # x, y, xdot, ydot
    goal = data[planIdx, 8:12]
    start_yaw = np.sign(int(start[-1] > 0) - 0.5) * np.arccos(np.dot(start[2:4] / np.linalg.norm(start[2:4]), [1, 0])) # get yaw from xdot, ydot
    goal_yaw = np.sign(int(goal[-1] > 0)- 0.5) * np.arccos(np.dot(goal[2:4] / np.linalg.norm(goal[2:4]), [1, 0]))

    for i, pdef in enumerate(pdefs):
        pdef.clear()
        plan(pdef, initState=np.append(start[:2], start_yaw), goalState=np.append(goal[:2], goal_yaw))

        if pdef.getProblemDefinition().hasExactSolution():
            if pdef.getPlanner().bestCost().value() > minCostThreshold:
                sol = np.loadtxt(StringIO(pdef.getProblemDefinition().getSolutionPath().printAsMatrix()))

                nIterations[i, nCompletedTests] = pdef.getPlanner().numIterations()
                pathCosts[i, nCompletedTests] = pdef.getPlanner().bestCost().value()
                
                if plotPaths:
                    fig1 = plt.figure(figsize=(10,6), dpi=80)
                    ax1 = fig1.add_subplot(111, aspect='equal')
                    ax1.imshow(occGrid, extent=[0, gridSize-1, 0, gridSize-1], cmap='gray')
                    ax1.scatter(y[:,0] * gridSize, y[:,1] * gridSize, color="green", s=70, alpha=0.1)
                    ax1.quiver(sol[:, 0] * gridSize, sol[:, 1] * gridSize, np.cos(sol[:,2]), np.sin(sol[:,2]), color="purple", scale=8.0, width=0.015, alpha=0.9)
                    ax1.quiver(start[0] * gridSize, start[1] * gridSize, start[2], start[3], color="red", scale=8.0, width=0.015) # init
                    ax1.quiver(goal[0] * gridSize, goal[1] * gridSize, goal[2], goal[3], color="blue", scale=8.0, width=0.015) # goal
                    plt.show()
            else:
                nCompletedTests -= 1
                break 
        else:
            nCompletedTests -= 1
            break
    
    nCompletedTests += 1


In [ ]:
print(nIterations)
print('Mean num iterations: {}, std: {}'.format(np.mean(nIterations, axis=1), np.std(nIterations, axis=1)))
print('Mean path cost: {}'.format(np.mean(pathCosts, axis=1)))